<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/test_runs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Welcome to Okareo!

Get your API token from [https://app.okareo.com/](https://app.okareo.com/) and set it in the cell below. 👇
   (Note: You will need to register first.)



In [ ]:
OKAREO_API_KEY = "<YOUR-OKAREO-API-TOKEN>"

%pip install okareo

We're going to set up a simple classification task that will score a model on how accurately it can classify different scenarios. The setup will have three parts:

1. A "model" that classifies scenarios as either "pricing," "returns," or "complaints"
2. A set of scenarios to test the model on
3. An evaluation of the model

## The model

In [ ]:
# Load all of the necessary libraries from Okareo
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, SeedData
from okareo.model_under_test import CustomModel, ModelInvocation

# Create an instance of the Okareo client
okareo = Okareo(OKAREO_API_KEY)

# Define a simplified "model" that will be used used for classification
# The model takes in a scenario and returns a predicted class
class ClassificationModel(CustomModel):
    # Callable to be applied to each scenario in the scenario set
    def invoke(self, input: str) -> ModelInvocation:
        # Rules based on keywords to determine the intent of the input
        # This is the logic of the "model" that will be used for classification
        if "how much" in input:
            prediction = "pricing"
        elif "return" in input:
            prediction = "returns"
        else:
            prediction = "complaints"
        
        # Return the prediction and the overall model response context
        return  ModelInvocation(model_prediction=prediction, model_output_metadata={"labels": prediction, "confidence": .99 }) # confidence is a placeholder

# Register the model with Okareo
# This will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier", model=ClassificationModel(name="Classification model"))

## The scenarios

In [ ]:
# Define a scenario set
# This is a collection of scenarios that will be used to test the model
scenario_set_create = ScenarioSetCreate(name="My Test Scenario Set", # Name of the scenario set
                                        # The data that will be used to test the model
                                        # Each SeedData object has an input (the scenario) and a result (the expected output)
                                        seed_data=[SeedData(input_="I want to send this product back", result="returns"),
                                                   SeedData(input_="my product is not working", result="complaints"),
                                                   SeedData(input_="how much is the warranty on the product", result="pricing"),
                                                   SeedData(input_="this product is having issues", result="complaints"),
                                                   SeedData(input_="I want to send this product back for a return", result="returns"),
                                                   SeedData(input_="how much is this product", result="pricing")])

# Create the scenario set
scenario = okareo.create_scenario_set(scenario_set_create)
scenario_id = scenario.scenario_id

## Evaluation

In [ ]:
# Run the test
# This will run the model on the scenarios in the scenario set
test_run_item = model_under_test.run_test(scenario=scenario_id, name="Intent Classifier Run", calculate_metrics=True)

# Generate a link back to Okareo for evaluation visualization
model_results = test_run_item.model_metrics.to_dict()
app_link = test_run_item.app_link
print(f"See results in Okareo: {app_link}")